In [1]:
import pickle
from matplotlib import pyplot as plt
import pandas as pd
from curate_kcat import *

from multiprocessing import Process,cpu_count,Manager

In [2]:
import curate_kcat_light_goslim_asc_step02 as step02
import curate_kcat_light_goslim_asc_step03 as step03

In [3]:
from curate_kcat_light_goslim_asc_step1 import *
results02 = pickle.load(open('../Results/smc_abc_light_step_02.pkl','rb'))

results03 = pickle.load(open('../Results/smc_abc_light_step_03.pkl','rb'))

ModuleNotFoundError: No module named 'curate_kcat_light_goslim_asc_step1'

In [ ]:
dffrac = pd.read_csv('../Results/protein_abundance_go_slim_level_uniq_asc.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')
dfomics = pd.read_csv('../proteomics/protein_abandance_mean.csv',index_col=0)
dfmws = pd.read_csv('../Results/protein_mws.csv',index_col=0)
dfmws = dfmws/1000

In [ ]:
test_conds = []
for cond_id in dftot.index:
    if cond_id not in dfpheno.index:
        test_conds.append(cond_id)
        dfpheno.loc[cond_id,'P3HB'] = 0
print(test_conds)

In [ ]:
model_go_pools_file = '../Results/template_ecModel_goslim_pools.pkl'
model_go_pools = pickle.load(open(model_go_pools_file,'rb'))
    
eModel_file = '../Results/template_eModel.pkl'
eModel = pickle.load(open(eModel_file,'rb'))

In [ ]:
step02_results = dict() # {cond_id:[]*100}
for kcat in results02.population:
    for cond_id in test_conds:
        r = step02.test_model(
                       model_go_pools,
                       kcat,          # kcats: dict() {(rxn_id, prot_met_id): kcat in h-1}
                       dftot,          # contains the total protein abandance, in the unit of gram protein/gram CDW
                       dfpheno,        # contains the P3HB specific synthetic rate
                       dffrac,
                       cond_id)
        step02_results[cond_id] = step02_results.get(cond_id,[]) + [r[1]]


In [ ]:
step03_results = dict() # {cond_id:[]*100}
for kcat in results03.population:
    for cond_id in test_conds:
        r = step03.test_model(#model_one_pool,      # json format
               eModel,
               kcat,          # kcats: dict() {(rxn_id, prot_met_id): kcat in h-1}
               dftot,          # contains the total protein abandance, in the unit of gram protein/gram CDW
               dfpheno,        # contains the P3HB specific synthetic rate
               cond_id,   # condition id
               dfomics,
               dfmws
                )
        step03_results[cond_id] = step03_results.get(cond_id,[]) + [r[1]]


In [ ]:
pickle.dump([step02_results,step03_results],open('../Results/validation_growth_on_unused_conditions.pkl','wb'))

In [ ]:
data1 = [step02_results[item] for item in test_conds]
data2 = [step03_results[item] for item in test_conds]
pos = np.arange(len(data1))*3

In [ ]:
plt.boxplot(data1,positions=pos)
plt.boxplot(data2,positions=pos+1)
plt.show()

### Prior models

In [ ]:
step02_results_prior = dict() # {cond_id:[]*100}
for kcat in results02.population_t0:
    for cond_id in test_conds:
        r = step02.test_model(
                       model_go_pools,
                       kcat,          # kcats: dict() {(rxn_id, prot_met_id): kcat in h-1}
                       dftot,          # contains the total protein abandance, in the unit of gram protein/gram CDW
                       dfpheno,        # contains the P3HB specific synthetic rate
                       dffrac,
                       cond_id)
        step02_results_prior[cond_id] = step02_results_prior.get(cond_id,[]) + [r[1]]


In [ ]:
step03_results_prior = dict() # {cond_id:[]*100}
for kcat in results03.population_t0:
    for cond_id in test_conds:
        r = step03.test_model(#model_one_pool,      # json format
               eModel,
               kcat,          # kcats: dict() {(rxn_id, prot_met_id): kcat in h-1}
               dftot,          # contains the total protein abandance, in the unit of gram protein/gram CDW
               dfpheno,        # contains the P3HB specific synthetic rate
               cond_id,   # condition id
               dfomics,
               dfmws
                )
        step03_results_prior[cond_id] = step03_results_prior.get(cond_id,[]) + [r[1]]


In [ ]:
pickle.dump([step02_results_prior,step03_results_prior],open('../Results/validation_growth_on_unused_conditions_prior.pkl','wb'))